In [ ]:
!conda install -y gdown
!pip install google-play-scraper
!pip install PySastrawi

/bin/bash: line 1: conda: command not found


In [ ]:
from google_play_scraper import Sort, reviews_all, reviews
import re
import pandas as pd
import numpy as np
import datetime as dt
import torch
import string
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix, classification_report
# from google_play_scraper import Sort, reviews_all, reviews
import nltk
# import calendar
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from wordcloud import WordCloud
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('popular')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
label = ['positif', 'netral', 'negatif']
# Load pre-trained model and tokenizer
model_name = "ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Example input text
input_text = "aku cinta indonesia"

# Tokenize input text
input_ids = tokenizer.encode(input_text, add_special_tokens=True, max_length=1024, truncation=True, return_tensors='pt').to(device)

# Perform inference
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits

# Get predicted label
predicted_label = torch.argmax(logits, dim=1).item()
print("Predicted label:",label[predicted_label])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Predicted label: positif


In [ ]:
import re
from nltk.corpus import stopwords

def text_preprocessing(text):
  """
  Fungsi untuk melakukan text preprocessing dengan langkah-langkah:
  1. Cleansing
  2. Case Folding
  3. Tokenizing
  4. Filtering

  Args:
      text (str): Teks yang ingin diproses.

  Returns:
      list: List of kata-kata yang telah diproses.
  """

  # 1. Cleansing
  text = re.sub(r"[^\w\s]", "", text) # Menghapus karakter non-alfabetis
  text = text.lower() # Case folding

  # 2. Tokenizing
  tokens = text.split()

  # 3. Filtering
  # Mengganti stopwords bahasa Inggris dengan bahasa Indonesia
  stop_words = set(stopwords.words("indonesian"))
  filtered_tokens = [token for token in tokens if token not in stop_words]

  return ' '.join(filtered_tokens)

# Contoh penggunaan
text = "Contoh teks untuk preprocessing. Teks ini berisi tanda baca, simbol, dan kata-kata yang tidak penting."

preprocessed_text = text_preprocessing(text)

print(preprocessed_text)


contoh teks preprocessing teks berisi tanda baca simbol katakata


In [ ]:
import pandas as pd

data = pd.read_csv('https://docs.google.com/spreadsheets/d/1xsx3f4lJBswSe39yxzvl7_k6M1OTSnSrJt5lpmIEyDA/gviz/tq?tqx=out:csv')

In [ ]:
data.head()

,publishedAt,authorDisplayName,textDisplay,likeCount
0,2023-11-14T03:32:13Z,@yanglucu-lucu4280,Partai PAN SI ZUL NIH. BELUM JADI PRESIDEN AJA...,0
1,2023-10-30T08:37:26Z,@umbetchannel2844,mohon game-game juga di tutupanakku saja sampa...,0
2,2023-10-28T07:57:03Z,@kageming8926,jujur setelah tiktok shop di tutup Uang semaki...,0
3,2023-10-27T08:21:18Z,@bayunugroho9197,Pemerintah cuma minta urus ijin ya kali nyedot...,0
4,2023-10-27T02:43:15Z,@agungabdillah4557,Menutup tiktokshop supaya pasar tanah abang ra...,0


In [ ]:
data['clean_review'] = [text_preprocessing(data['textDisplay'][i]) for i in range(len(df))]

In [ ]:
data.head()

,publishedAt,authorDisplayName,textDisplay,likeCount,clean_review
0,2023-11-14T03:32:13Z,@yanglucu-lucu4280,Partai PAN SI ZUL NIH. BELUM JADI PRESIDEN AJA...,0,partai pan si zul nih presiden aja udah mencet...
1,2023-10-30T08:37:26Z,@umbetchannel2844,mohon game-game juga di tutupanakku saja sampa...,0,mohon gamegame tutupanakku sakit gara2 game
2,2023-10-28T07:57:03Z,@kageming8926,jujur setelah tiktok shop di tutup Uang semaki...,0,jujur tiktok shop tutup uang pekat
3,2023-10-27T08:21:18Z,@bayunugroho9197,Pemerintah cuma minta urus ijin ya kali nyedot...,0,pemerintah urus ijin ya kali nyedot uang trily...
4,2023-10-27T02:43:15Z,@agungabdillah4557,Menutup tiktokshop supaya pasar tanah abang ra...,0,menutup tiktokshop pasar tanah abang ramai kek...


In [ ]:
output=[]
for i, val in data.iterrows():
    input_ids = tokenizer.encode(val['clean_review'], add_special_tokens=True,  max_length=512, truncation=True,  return_tensors='pt').to(device)
    with torch.no_grad():
        try:
            outputs = model(input_ids)
        except:
            print(val['clean_review'])
            break
        logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()
    output.append(label[predicted_label])

In [ ]:
data['label'] = output
data.head()

,publishedAt,authorDisplayName,textDisplay,likeCount,clean_review,label
0,2023-11-14T03:32:13Z,@yanglucu-lucu4280,Partai PAN SI ZUL NIH. BELUM JADI PRESIDEN AJA...,0,partai pan si zul nih presiden aja udah mencet...,negatif
1,2023-10-30T08:37:26Z,@umbetchannel2844,mohon game-game juga di tutupanakku saja sampa...,0,mohon gamegame tutupanakku sakit gara2 game,negatif
2,2023-10-28T07:57:03Z,@kageming8926,jujur setelah tiktok shop di tutup Uang semaki...,0,jujur tiktok shop tutup uang pekat,netral
3,2023-10-27T08:21:18Z,@bayunugroho9197,Pemerintah cuma minta urus ijin ya kali nyedot...,0,pemerintah urus ijin ya kali nyedot uang trily...,negatif
4,2023-10-27T02:43:15Z,@agungabdillah4557,Menutup tiktokshop supaya pasar tanah abang ra...,0,menutup tiktokshop pasar tanah abang ramai kek...,negatif


In [ ]:
data.to_csv('/content/dataset_yt_febri_fix_label.csv', index=False)